# Import Libraries and Device Info

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [24]:
# Check the devices that are available for computation

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14888441873328827955
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7279392031149924571
physical_device_desc: "device: XLA_CPU device"
]


In [25]:
print(tf.__version__)

2.3.1


# Exploiratory Data Analysis

In [26]:
train_dir = './Datasets/train'
test_dir = './Datasets/test'

ImageDataGenerator generates batches of tensor image data with real-time data augmentation.

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
data = ImageDataGenerator(rescale=1/255.0)

In [29]:
train_generator = data.flow_from_directory(directory=train_dir, target_size=(256,256), classes=['cats', 'dogs'], class_mode='binary', batch_size=20)

Found 20000 images belonging to 2 classes.


In [30]:
train_generator.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [31]:
cv_generator = data.flow_from_directory(directory=test_dir, target_size=(256,256), classes=['cats', 'dogs'], class_mode='binary', batch_size=20)

Found 5000 images belonging to 2 classes.


In [32]:
print("The data size in each Train batch is:", len(train_generator))
print("The data size in each Cross Validation batch is:", len(cv_generator))

The data size in each Train batch is: 1000
The data size in each Cross Validation batch is: 250


# Build CNN Model

In [33]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [34]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='he_uniform', input_shape=(256,256,3)))
model.add(MaxPool2D(2,2))

model.add(Flatten())
model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(units=1, activation='sigmoid'))

In [35]:
from tensorflow.keras.optimizers import SGD

In [36]:
optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 524288)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               67108992  
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 67,110,017
Trainable params: 67,110,017
Non-trainable params: 0
_________________________________________________________________


#  Fitting Model

In [38]:
records = model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=cv_generator, validation_steps=len(cv_generator), verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1000/1000 [==============================] - 392s 392ms/step - loss: 0.7284 - accuracy: 0.5903 - val_loss: 0.6318 - val_accuracy: 0.6216
Epoch 2/10
1000/1000 [==============================] - 311s 311ms/step - loss: 0.6003 - accuracy: 0.6660 - val_loss: 0.6733 - val_accuracy: 0.6542
Epoch 3/10
1000/1000 [==============================] - 313s 313ms/step - loss: 0.4755 - accuracy: 0.7686 - val_loss: 0.6893 - val_accuracy: 0.6634
Epoch 4/10
1000/1000 [==============================] - 313s 313ms/step - loss: 0.3015 - accuracy: 0.8717 - val_loss: 0.7424 - val_accuracy: 0.6664
Epoch 5/10
1000/1000 [==============================] - 312s 312ms/step - loss: 0.1520 - accuracy: 0.9468 - val_loss: 1.0674 - val_accuracy: 0.6684
Epoch 6/10
1000/1000 [==============================] - 312s 312ms/step - loss: 0.0806 - accuracy: 0.9765 - val_loss: 1.4007 - val_accuracy: 0.6706
Epoch 7/10
1000/1000 [==============